# Use Pre-Trained Model

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import torch
import tomli
import matplotlib.pyplot as plt
from os import environ
from pathlib import Path
from buildings_bench import load_torch_dataset
from buildings_bench.models import model_factory

/global/homes/n/nrushad/.conda/envs/BuildingsBenchEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class DataHandler:
    def __init__(self): pass

    def load_dataset(self, dataset_name, scaler_transform):
        dataset = load_torch_dataset(
            name=dataset_name,
            apply_scaler_transform=scaler_transform,
            scaler_transform_path=Path(environ["TRANSFORM_PATH"])
        )
        return dataset

    def create_dataloader(self, building_dataset):
        dataloader = torch.utils.data.DataLoader(
                        building_dataset,
                        batch_size=360,
                        shuffle=False)
        return dataloader

In [42]:
import torch.nn as nn

class MyModel():
    def __init__(self,
                 hidden_size=3,
                 context_len=168,
                 pred_len=24,
                 continuous_loads=True,
                 **args):

        self.context_len=context_len
        self.hidden_size=hidden_size
        self.pred_len=pred_len
        self.continuous_loads=continuous_loads


        """Init method for MyModel.

        Args:
            hidden_size (int): size of hidden layer
            context_len (int): length of context window
            pred_len (int): length of prediction window
            continuous_loads (bool): whether this model uses continuous load values
        """
        # super().__init__(context_len, pred_len, continuous_loads)

        # Our model will be a simple MLP with two hidden layers
        # and will output two values (mean and std dev) for each hour.
        self.mlp = nn.Sequential(
            nn.Linear(context_len, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, pred_len*2)
        )

    def forward(self, x):
        """
        `x` is a dictionary with the following keys:

        ```
        'load': torch.Tensor,               # (batch_size, seq_len, 1)
        'building_type': torch.LongTensor,  # (batch_size, seq_len, 1)
        'day_of_year': torch.FloatTensor,   # (batch_size, seq_len, 1)
        'hour_of_day': torch.FloatTensor,   # (batch_size, seq_len, 1)
        'day_of_week': torch.FloatTensor,   # (batch_size, seq_len, 1)
        'latitude': torch.FloatTensor,      # (batch_size, seq_len, 1)
        'longitude': torch.FloatTensor,     # (batch_size, seq_len, 1)
        ```

        This model only uses the 'load'.
        """
        # (batch_size, self.context_len)
        x = x['load'][:, :self.context_len, 0]
        out = self.mlp(x)  # (batch_size, self.pred_len*2)
        return out.view(-1, self.pred_len, 2) # (batch_size, self.pred_len, 2)

    def loss(self, x, y):
        """
        Args:
            x (torch.Tensor): preds of shape (batch_size, seq_len, 2)
            y (torch.Tensor): targets of shape (batch_size, seq_len, 1)
        Returns:
            loss (torch.Tensor): scalar loss
        """
        return nn.functional.gaussian_nll_loss(x[:, :, 0].unsqueeze(2), y,
                                       nn.functional.softplus(x[:, :, 1].unsqueeze(2))**2)

    def predict(self, x):
        """
        Args:
            x (Dict): dictionary of input tensors
        Returns:
            predictions (torch.Tensor): of shape (batch_size, pred_len, 1)
            distribution_parameters (torch.Tensor): of shape (batch_size, pred_len, -1)
        """
        out = self.forward(x)
        means = out[:, :, 0].unsqueeze(2)
        stds = nn.functional.softplus(out[:, :, 1].unsqueeze(2))
        return means, torch.cat([means, stds], dim=2)

    # def unfreeze_and_get_parameters_for_finetuning(self):
    #     """For transfer learning."""
    #     return self.parameters()

    # def load_from_checkpoint(self, checkpoint_path):
    #     """Describe how to load model from a checkpoint."""
    #     self.load_state_dict(torch.load(checkpoint_path)['model'])

In [43]:
def main(model_name, dataset_name, scaler_transform, device):
    handler = DataHandler()
    dataset = handler.load_dataset(dataset_name, scaler_transform)    

    model = MyModel()

    target_list = []
    prediction_list = []


    for epoch in range(10):
    
        for building_id, building_dataset in dataset:
            
            inverse_transform = building_dataset.datasets[0].load_transform.undo_transform
            dataloader = handler.create_dataloader(building_dataset)
            
            for batch in dataloader:
                for key, value in batch.items():
                    batch[key] = value.to(device)
                    x = model.predict(batch)
                    
                    
    
                # with torch.cuda.amp.autocast(enabled=(device == 'cuda:0')):
                #     outputs, h0, c0 = model(batch, h0, c0) 
                #     print(outputs)

In [44]:
if __name__ == "__main__":

    DATASET = 'ideal' # Electricity | SMART | Ideal | lcl
    DEVICE = 'cuda:0'
    MODEL = 'TransformerWithGaussian-L' # TransformerWithGaussian-S | TransformerWithGaussian-M
    CHECKPOINT = 'Transformer_Gaussian_L.pt'
    SCALER_TRANSFORM = 'boxcox'

    PATH = '/pscratch/sd/n/nrushad'
    environ["PATH"] = PATH
    
    environ["REPO_PATH"] = f'{PATH}/BuildingsBenchTutorial/BuildingsBench/'
    environ["BUILDINGS_BENCH"] = f'{PATH}/Dataset'
    environ["TRANSFORM_PATH"] = f'{PATH}/Dataset/metadata/transforms'
    
    main(MODEL, DATASET, SCALER_TRANSFORM, DEVICE)

(tensor([[[ 0.0815],
         [-0.1070],
         [ 0.1185],
         ...,
         [-0.4695],
         [ 0.4544],
         [ 0.3193]],

        [[-0.0911],
         [-0.2044],
         [-0.0157],
         ...,
         [-0.3615],
         [ 0.3094],
         [ 0.5264]],

        [[-0.1047],
         [-0.2121],
         [-0.0263],
         ...,
         [-0.3530],
         [ 0.2980],
         [ 0.5426]],

        ...,

        [[ 0.1109],
         [-0.0905],
         [ 0.1414],
         ...,
         [-0.4879],
         [ 0.4791],
         [ 0.2840]],

        [[ 0.2364],
         [-0.0196],
         [ 0.2390],
         ...,
         [-0.5664],
         [ 0.5846],
         [ 0.1334]],

        [[ 0.0700],
         [-0.1135],
         [ 0.1096],
         ...,
         [-0.4623],
         [ 0.4448],
         [ 0.3330]]], grad_fn=<UnsqueezeBackward0>), tensor([[[ 0.0815,  0.5623],
         [-0.1070,  0.4678],
         [ 0.1185,  0.9373],
         ...,
         [-0.4695,  0.7502],
        

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)